该脚本作用：
将泰迪内推平台任务下发，将所有简单的一级页面内存储到数据库中。

In [2]:
import redis
import requests

# 岗位信息

## 获取岗位信息列表

In [3]:
# 岗位API
job_api = 'https://www.5iai.com/api/enterprise/job/public/es'
# 设定Get请求参数
job_params = {
    'pageSize': '2000',
    'pageNumber': 1
}

In [4]:
# 发送GET请求
job_response = requests.get(job_api, params=job_params)

In [5]:
# 将响应内容转化为字典
job_dict = job_response.json()
# 获取岗位信息列表
job_list = job_dict['data']['content']
# 查看岗位数量
print('共获取%d个岗位' % len(job_list))

共获取1574个岗位


## 将一级页面内容存储到数据库

In [6]:
all_job_id = [i['id'] for i in job_list]  # 获取招聘信息ID
job_name = [i['enterpriseExtInfo']['shortName'] for i in job_list]
job_positionName = [i['positionName'] for i in job_list]
job_educationalRequirements = [i['educationalRequirements'] for i in job_list]
job_count = [i['count'] for i in job_list]
job_econKind = [i['enterpriseExtInfo']['econKind'] for i in job_list]
job_minimumWage = [i['minimumWage'] for i in job_list]
job_maximumWage =  [i['maximumWage'] for i in job_list]

In [7]:
import pymysql
# 连接数据库
conn = pymysql.connect(host='localhost',
                      port=3306,
                      database='tipdm_cup',
                      user='root',
                      passwd='123456')

In [8]:
# 创建游标
cursor = conn.cursor()

In [9]:
job_sql = "insert into job_1(job_id, job_name, job_positionName, job_educationalRequirements, job_count, job_econKind, job_minimumWage, job_maximumWage) Values" + ",".join([str(i) for i in list(zip(all_job_id, job_name, job_positionName, job_educationalRequirements, job_count, job_econKind, job_minimumWage, job_maximumWage))])

In [10]:
cursor.execute(job_sql)

1574

In [11]:
conn.commit()
cursor.close()
conn.close()

## 将岗位二级标签任务加入到Redis数据库中

In [12]:
# 连接Redis数据库
redis_conn = redis.Redis(host="192.168.128.128", port=6379, password="minglog")

In [13]:
# 可以放入服务器的Redis数据库中
redis_conn.sadd('all_job_list', *all_job_id)

1

In [14]:
redis_conn.close()

# 求职信息

## 获取求职信息列表

In [15]:
# 求职API
user_api = 'https://www.5iai.com/api/resume/baseInfo/public/es'
# 设定Get请求参数
user_params = {
    'pageSize': 99999,
    'pageNumber': 1
}

In [16]:
# 发送GET请求
user_response = requests.get(user_api, params=user_params)

In [17]:
# 将响应内容转化为字典
user_dict = user_response.json()
# 获取求职者信息列表
user_list =  user_dict['data']['content']

## 将一级页面内容存储到数据库

In [18]:
user_id = [i['id'] for i in user_list]
user_name = [i['username'] for i in user_list]
user_exp = [i['exp'] for i in user_list]
user_expectPosition = [i['expectPosition'] for i in user_list]
user_willSalaryEnd = [i['willSalaryEnd'] for i in user_list]
user_willSalaryStart = [i['willSalaryStart'] for i in user_list]

In [19]:
user_exp = ['无经验'  if i == None else i for i in user_exp]

In [20]:
import pymysql
# 连接数据库
conn = pymysql.connect(host='localhost',
                      port=3306,
                      database='tipdm_cup',
                      user='root',
                      passwd='123456')

In [21]:
# 创建游标
cursor = conn.cursor()

In [22]:
job_sql = "insert into user_1(user_id, user_name, user_exp, user_expectPosition, user_willSalaryEnd, user_willSalaryStart) Values" + ",".join([str(i) for i in list(zip(user_id, user_name, user_exp, user_expectPosition, user_willSalaryEnd, user_willSalaryStart))])

In [23]:
cursor.execute(job_sql)

10911

In [24]:
conn.commit()
cursor.close()
conn.close()

## 将求职二级标签任务加入到Redis数据库中

In [25]:
# 连接Redis数据库
redis_conn = redis.Redis(host="192.168.128.128", port=6379, password="minglog")

In [26]:
# 可以放入服务器的Redis数据库中
redis_conn.sadd('all_user_list', *user_id)

0

In [27]:
redis_conn.close()